In [ ]:
import pandas as pd
import dateutil.parser as dp
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import os
import glob
from collections import OrderedDict
import numpy as np
import scipy

In [ ]:
frameworks = ['fuzzware', 'halfuzz', 'libAFL_halfuzz', 'SAFIREFUZZ']

In [ ]:
targets = ['6lowpan_rx', '6lowpan_tx', 'nxp http', 'samr21 http_eth', 'p2im plc',  'p2im_drone', 'st-plc', 'wycinwyc', 'stm32_tcp_echo_client', 'stm32_tcp_echo_server', 'stm32_udp_echo_client', 'stm32_udp_echo_server']

In [ ]:
# readall data
all_data = {}
for t in targets:
    all_data[t] = {}
    for fw in frameworks:
        print(f"Processing {t} for {fw}")

        all_data[t][fw] = {}
        with open(f'./SAFIRE experiments - {fw}.csv', 'r') as f:
            data = f.read()
        # inefficient but whatever:
        data = [d for d in data.split('\n') if t in d]
        print(data)

        assert(len(data)==5)
        all_data[t][fw] = [float(d.split(',')[2].replace('"','')) for d in data]
print(all_data)

In [ ]:
for fw in frameworks:
    if fw == 'SAFIREFUZZ':
        continue
    found_violation = False
    print(f'=== Comparing SafireFuzz with {fw} ===')
    
    for t in targets:

        safire_maxs = all_data[t]['SAFIREFUZZ']
        target_maxs = all_data[t][fw]
        mannu = scipy.stats.mannwhitneyu(safire_maxs, target_maxs, method='auto')
        if mannu.pvalue > 0.05:
            print(f'Offending P-Value while comparing against {t}: {mannu.pvalue:.03f}')
            found_violation=True
    if found_violation is False:
        print('No offending P-Values found')
    print()

In [ ]:
import pickle
with open('execs_aggregrated', 'wb') as f:
    f.write(pickle.dumps(all_data))